In [1]:
import numpy as np
import pandas as pd

In [2]:
# Loading datasets
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

# Reasoning

After abandoning this project for a long time, I wanted to finish it. However, despite my notes, I was not really sure what my thought process was when I last worked on it. Therefore, I wanted to start the project again, incorporating my old code wherever I could, but with a clearer understanding of what I wanted to do and how I would achieve it. I started again on 5/15/24 and aim to finish by 5/20/24. The steps I intend to complete are:

- Beginning model using rule-based constraints
- decision tree
- random forest
- cleaning and more thorough EDA
- rerun dt and rf models
- feature engineering
- rerun dt and rf models
- hyperparamter tuning
- final dt and rf models

I also intend to submit my models to kaggle with an aim of getting to at least 97.5% accuracy. I am unsure how many times I will submit, but hopefully at least once per data improvement stage.

In [3]:
# Overall

overall_survival_rate = sum(train['Survived'])/len(train['Survived'])
print(f'The overall survival rate for the test set: {overall_survival_rate:.2%}')

The overall survival rate for the test set: 38.38%


In [4]:
# men vs female

women = train.loc[train['Sex'] == 'female']['Survived']
women_survival_rate = sum(women)/len(women)
print(f'The percentage of women that survived: {women_survival_rate:.2%}')

men = train.loc[train['Sex'] == 'male']['Survived']
men_survival_rate = sum(men)/len(men)
print(f'The percentage of men that survived: {men_survival_rate:.2%}')

The percentage of women that survived: 74.20%
The percentage of men that survived: 18.89%


In [8]:
# young vs old

young = train.loc[train['Age'] < 16]['Survived']
young_survival_rate = sum(young)/len(young)
print(f'The percentage of young passengers that survived: {young_survival_rate:.2%}')

old = train.loc[train['Age'] >= 16]['Survived']
old_survival_rate = sum(old)/len(old)
print(f'The percentage of old passengers that survived: {old_survival_rate:.2%}')

The percentage of young passengers that survived: 57.97%
The percentage of old passengers that survived: 38.76%


In [22]:
# Women and Children

women_and_children = train.loc[(train['Age'] < 16) | (train['Sex'] == 'female')]['Survived']
women_and_children_survival_rate = sum(women_and_children)/len(women_and_children)
print(f'The percentage of women and children that survived: {women_and_children_survival_rate:.2%}')


not_women_and_children = train.loc[~((train['Age'] < 16) | (train['Sex'] == 'female'))]['Survived']
not_women_and_children_survival_rate = sum(not_women_and_children)/len(not_women_and_children)
print(f'The percentage of older men that survived: {not_women_and_children_survival_rate:.2%}')

The percentage of women and children that survived: 71.75%
The percentage of older men that survived: 16.39%


In [20]:
# TODO create confusion matrix for results

print(len(train))

891


In [ ]:
# TODO use filter for test data and upload to kaggle for baseline